# Integration with External Modeling Code (PHOEBE)

* **Last Updated**: March 7, 2024,
* **lcviz version**: 0.2.0

In [ ]:
# If phoebe is not installed, install phoebe:
try:
    import phoebe
except ImportError:
    !pip install phoebe
    import phoebe

In [ ]:
from lightkurve import search_lightcurve

from lcviz import LCviz

lc = search_lightcurve("KIC 2835289", mission="Kepler", cadence="long", quarter=10).download()

lcviz = LCviz()
lcviz.load_data(lc)
lcviz.show()

In [ ]:
flatten = lcviz.plugins['Flatten']
flatten.open_in_tray()

In [ ]:
flatten.unnormalize = True
lc = flatten.flatten()

In [ ]:
flatten.close_in_tray()

In [ ]:
ephem = lcviz.plugins['Ephemeris']
ephem.open_in_tray()

In [ ]:
#ephem.method = 'Box Least Squares'

In [ ]:
ephem.adopt_period_at_max_power()

In [ ]:
ephem.period *= 2

In [ ]:
ephem.wrap_at = 0.5

In [ ]:
#ephem.t0 = 0.016

In [ ]:
# could be available from phoebe.default_binary_from_lcviz or also from the export plugin in lcviz (either
# code could either live in phoebe or an extra package that links the two and registers the ability in both codes
def default_binary_from_lcviz(lcviz, ephem_map={'default': 'binary'}, b=None):  
    import numpy as np
    if b is None:
        b = phoebe.default_binary()

    # datasets
    ref_time = None
    for data in lcviz.app.data_collection:
        # TODO: ignore non flux-vs-time entries
        label = data.label
        lc = lcviz.get_data(data.label)
        flux_col = lcviz.plugins['Flux Column'].flux_column.selected
        fluxes = lc[flux_col].value
        mask = np.isfinite(fluxes)
        b.add_dataset('lc',
                      times=lc['time'].value[mask], fluxes=fluxes[mask],
                      compute_phases=phoebe.linspace(0,1,201),
                      pblum_mode='dataset-scaled',
                      dataset=label.replace(' ', '_'))
        if ref_time is None and data.meta.get('reference_time'):
            ref_time = data.meta.get('reference_time').value
    
    if ref_time is None:
        ref_time = 0.0

    b.set_value(qualifier='t0', context='system', value=ref_time)
    
    # ephemerides
    ephem_plg = lcviz.plugins.get('Ephemeris')
    for lcviz_comp, phoebe_comp in ephem_map.items():
        # TODO: support for rotation periods, validate phoebe_comp
        # TODO: support for reftime in setting t0_supconj (and use that for t0@system)
        ephem = ephem_plg.ephemerides.get(lcviz_comp, {})
        b.set_value(qualifier='period', component=phoebe_comp, context='component', value=ephem.get('period', 1.0))
        b.set_value(qualifier='t0_supconj', component=phoebe_comp, context='component', value=ephem.get('t0', 0.0)+ref_time)
        b.set_value(qualifier='dpdt', component=phoebe_comp, context='component', value=ephem.get('dpdt', 0.0))  # TODO: units


    return b

In [ ]:
%matplotlib inline
b = default_binary_from_lcviz(lcviz)

In [ ]:
b.plot(x='phases', size=0.01, show=True)

In [ ]:
#b.add_solver('estimator.lc_geometry')
#b.run_solver()
#b.flip_constraint('requivsumfrac', solve_for='sma@binary')
#b.flip_constraint('teffratio', solve_for='teff@secondary')
#b.adopt_solution()

In [ ]:
b.run_compute()

In [ ]:
b.plot(x='phases', size=0.01, show=True)

In [ ]:
# definitely could be integrated into phoebe as b.to_lightkurve(dataset, model=None, ...)
def phoebe_model_to_lightkurve(b, model='latest'):
    model_ps = b.filter(model=model, context='model')
    times = model_ps.get_value(qualifier='times')
    fluxes = model_ps.get_value(qualifier='fluxes')
    return times, fluxes

In [ ]:
model_lc = phoebe_model_to_lightkurve(b)

In [ ]:
lcviz.load_data(model_lc)
# plot as line, not scatter, in time-viewer (can we support phase-sorted line in phase-viewer?)